In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
code_departement = "093"
compare_with_idf = True

# Imports & setup

In [3]:
import logging

import geopandas as gpd
import matplotlib.pyplot as plt

from potentiel_solaire.constants import DATA_FOLDER, CRS
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_buildings_of_interest
#from potentiel_solaire.aggregate import aggregate_solar_potential_by

logging.getLogger().setLevel("WARNING")

In [4]:
path_results = DATA_FOLDER / f"{code_departement}_pipeline_results.gpkg"

# Recupération des resultats intermédiaires

### Etablissements scolaires


In [5]:
schools_establishments = gpd.read_file(path_results, layer="schools_establishments")
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

Nb d'établissements scolaires: 1130


### Zones d'éducations

In [6]:
educational_zones = gpd.read_file(path_results, layer="educational_zones")
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

Nb de zones d'éducations:  1088


### Bâtiments

In [7]:
bd_topo_path = extract_bd_topo(code_departement=code_departement)
codes_commune = schools_establishments["code_commune"].unique()
communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(CRS)
communes = communes[communes.code_insee.isin(codes_commune)]
geom_of_interest = communes.dissolve()[["geometry"]]
buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=CRS
)

/home/kelu/projets/13_potentiel_solaire/.venv/lib/python3.10/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
/home/kelu/projets/13_potentiel_solaire/.venv/lib/python3.10/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
/home/kelu/projets/13_potentiel_solaire/.venv/lib/python3.10/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
/home/kelu/projets/13_potentiel_solaire/.venv/lib/python3.10/site-packages/pyogrio/core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
/home/kelu/projets/13_potentiel_solaire/.venv/lib/python3.10/site-packages/geopandas/io/file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_numerotee_ou_nommee', 'iti

### Rattachement des bâtiments scolaires

In [8]:
schools_buildings = gpd.read_file(path_results, layer="schools_buildings")
nb_schools_buildings = schools_buildings.shape[0]
print(f"Nb de batiments rattachés à des écoles: {nb_schools_buildings}")
# v0: Nb de batiments rattachés à des écoles: 3731

Nb de batiments rattachés à des écoles: 7151


### Calcul du potentiel solaire

In [9]:
solar_potential_of_schools_buildings = gpd.read_file(path_results, layer="solar_potential_of_schools_buildings")

# Checks sur la qualité des données & calculs

In [ ]:
nb_schools_with_buildings = len(schools_buildings.identifiant_de_l_etablissement.unique())
print("Nb d'établissements scolaires avec des batiments: {} ({}%)".format(
    nb_schools_with_buildings,
    round(100 * nb_schools_with_buildings / nb_schools)
))
# V0: Nb d'établissements scolaires avec des batiments: 445 (39%)
# ID MinEducation : Nb d'établissements scolaires avec des batiments: 1103 (98%)

Nb d'établissements scolaires avec des batiments: 1103 (98%)


In [ ]:
missing_schools_establishments = schools_establishments.merge(
    schools_buildings.drop_duplicates(subset=["identifiant_de_l_etablissement"])[["identifiant_de_l_etablissement", "cleabs_zone"]],
    on="identifiant_de_l_etablissement",
    how="left"
)
missing_schools_establishments = missing_schools_establishments[missing_schools_establishments["cleabs_zone"].isna()]
nb_missing_schools_establishments = missing_schools_establishments.shape[0]
print("Nb d'établissements scolaires sans batiment: {} ({}%)".format(
    nb_missing_schools_establishments,
    round(100 * nb_missing_schools_establishments / nb_schools)
))

# V0: Nb d'établissements scolaires sans batiment: 685 (61%)
# ID MinEducation :  Nb d'établissements scolaires sans batiment: 27 (2%)

Nb d'établissements scolaires sans batiment: 27 (2%)


In [ ]:
missing_educational_zones = educational_zones.merge(
    schools_buildings.drop_duplicates(subset=["cleabs_zone"])[["cleabs_zone"]],
    left_on="cleabs",
    right_on="cleabs_zone",
    how="left"
)
missing_educational_zones = missing_educational_zones[missing_educational_zones["cleabs_zone"].isna()]
missing_educational_zones.drop(columns=["cleabs_zone"], inplace=True)

nb_missing_educational_zones = len(missing_educational_zones.cleabs.unique())
print("Nb de zones d'educations non rattachées à un établissement scolaire: {} ({}%)".format(
    nb_missing_educational_zones,
    round(100 * nb_missing_educational_zones / nb_educational_zones)
))

#V0: Nb de zones d'educations non rattachées à un établissement scolaire: 643 (59%)
# ID MinEducation Nb de zones d'educations non rattachées à un établissement scolaire: 147 (14%)

Nb de zones d'educations non rattachées à un établissement scolaire: 147 (14%)


In [ ]:
missing_buildings = gpd.sjoin(
        buildings,
        missing_educational_zones,
        how='inner',
        predicate='intersects',
        lsuffix='bat',
        rsuffix='zone'
    )
nb_missing_buildings = missing_buildings.shape[0]
print(f"Nb de batiments sur des zones non rattachées: {nb_missing_buildings}")

# v0: Nb de batiments sur des zones non rattachées: 2451
# ID MinEducation: 581

Nb de batiments sur des zones non rattachées: 581


In [ ]:
school_buildings_only_in_educational_zones = gpd.overlay(
    solar_potential_of_schools_buildings,
    educational_zones[["geometry"]],
    how="intersection",
).to_crs(epsg=6933)

total_floor_area = solar_potential_of_schools_buildings["surface_totale_au_sol"].sum()
total_floor_area_only_in_educational_zones = school_buildings_only_in_educational_zones.area.sum()
area_not_in_educational_zones = total_floor_area - total_floor_area_only_in_educational_zones
print(f"{total_floor_area:,.0f} m² de surface au sol pour tous les batiments scolaires, dont {area_not_in_educational_zones:,.0f} m² ({100*area_not_in_educational_zones/total_floor_area:.0f}%) ne sont pas dans une zone d'education")
#V0: 1,671,571 m² de surface au sol pour tous les batiments scolaires, dont 198,644 m² (12%) ne sont pas dans une zone d'education
# ID MinEducation 3,725,023 m² de surface au sol pour tous les batiments scolaires, dont 373,259 m² (10%) ne sont pas dans une zone d'education

3,725,023 m² de surface au sol pour tous les batiments scolaires, dont 373,259 m² (10%) ne sont pas dans une zone d'education
